In [ ]:
import dciclient.oo
from datetime import datetime
client = dciclient.oo.DCIClient()

In [ ]:
openstack_team = client.teams.first(where='name:OpenStack')

In [ ]:
import pandas as pd
datelist = pd.date_range(start='2016/01/01', end=pd.datetime.today(), freq='M').tolist()

month_idx = 0

teams = {}

for job in client.jobs.list(sort="created_at", where='status:success'):
    if openstack_team.id() != job.team.parent_id():
        continue
    if not job.status != 'success':
        continue
    if not job.topic.name().startswith('OSP'):
        continue
    if job.team.external() is False:
        continue
    if job.team.name() not in teams:
        teams[job.team.name()] = [0 for _ in datelist]

    try:
        while not (
                datelist[month_idx:][0].year == job.created_at().year and 
                datelist[month_idx:][0].month == job.created_at().month):
            month_idx += 1
        teams[job.team.name()][month_idx] += 1
    except IndexError: # TODO: to remove
        pass

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.figure(figsize=(9,9))
for team in teams:
    plt.plot_date(datelist, teams[team], '.-', label=team)
plt.xlabel('Month')
plt.ylabel('Number of successful runs')
plt.title('DCI runs per month')
plt.legend()
plt.show()